# Notebook to filter Nashville GeoJSON file

The neighborhood information provided by Zillow provides a lot of different neighborhoods.  In fact, it even includes some neightborhoods outside the Davidson county area. Currently, we have been parsing through the relevant boundaries manually in Javascript in the HTML file, but in this notebook we'll create a new geoJSON file only containing the neighborhoods we wish to display.

In [1]:
import pygeoj
import pandas as pd

In [2]:
neighborhoods = pygeoj.load(filepath="Zillow_Neighborhoods.geojson")
davidson = pd.read_csv("davidson_counties.csv", dtype=str, usecols=['RegionID'])

In [3]:
print("Our original geoJSON file has %s neighborhoods." %len(neighborhoods))
print("Our Davidson County file contains %s neighborhoods." %len(davidson))

Our original geoJSON file has 798 neighborhoods.
Our Davidson County file contains 662 neighborhoods.


In [4]:
filtered_json = pygeoj.new()
for feature in neighborhoods:
    if(davidson['RegionID'].str.contains(feature.properties['RegionID']).any()):
         filtered_json.add_feature(feature)

In [5]:
print("Our new file will have information for %s neighborhoods." % len(filtered_json))

Our new file will have information for 662 neighborhoods.


In [6]:
filtered_json.add_all_bboxes()
filtered_json.update_bbox()
filtered_json.add_unique_id()
filtered_json.save("Updated_Zillow.geojson")